Patterns
========

This notebook involves the usage of general patterns - not necessarily object-oriented design patterns but simple patterns that might be useful for everyday problem solving an OAs.

## Sliding Window

Take some list of length `n`

In [2]:
var memory = Enumerable.Range(1, 10).ToArray().AsMemory();

for (int i = 0; i < memory.Length; i++)
{
    Console.Write(memory.Span[i]);
}

12345678910

Sometimes you want to iterate over slices of the array. You can do this with a `Span<T>`.

The formula for bounds checking in a standard `for` loop with a slice is $i < len - windowSize + 1$.

In [3]:
int windowSize = 7;

memory = Enumerable.Range(0, 10).ToArray().AsMemory();

for (int i = 0; i < memory.Length - windowSize + 1; i++)
{
    Console.WriteLine(memory.Span[i..(i + windowSize)].PrintSpan());
}

static string PrintSpan<T>(this Span<T> span)
{
    return string.Join(", ", span.ToArray());
}

0, 1, 2, 3, 4, 5, 6
1, 2, 3, 4, 5, 6, 7
2, 3, 4, 5, 6, 7, 8
3, 4, 5, 6, 7, 8, 9


### `for` loop with no body
Remember if you write a `for` loop without a body and put your body statement(s) next to your iteration statement, your body statement must come **before** the iteration statement because the iteration statement only runs _after_ the body has run at least once ("index `0`").

Take the following code that checks whether a `Span<T>` is strictly increasing:

In [15]:
var example = new int[] { 3, 4, 5, 6, 7, 8 };

static bool IsStrictlyIncreasing(Span<int> span)
{
    bool pass = true;
    for (int i = 0;
        pass && i < span.Length - 2 + 1;
        i++, Console.Write(i), pass &= (span[i + 1] > span[i]));

    return pass;
}

IsStrictlyIncreasing(example)

12345

Error: System.IndexOutOfRangeException: Index was outside the bounds of the array.
   at Submission#16.IsStrictlyIncreasing(Span`1 span)
   at Submission#16.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

See how the iteration starts at `1`, then right before `pass &= (span[i + 1] > span[i])` `i is 5`, then an `IndexOutOfRangeException` is thrown when `span[5 + 1]` is accessed on a `Span` of `Length == 6`?

Here's the fixed version:

In [17]:
var example = new int[] { 3, 3, 4 };

static bool IsStrictlyIncreasing(Span<int> span)
{
    bool pass = true;
    for (int i = 0;
        pass && i < span.Length - 2 + 1;
        pass &= (span[i + 1] > span[i]), i++);

    return pass;
}

IsStrictlyIncreasing(example)

False

You could also just use a `for` statement if you really don't want to have a body:

In [18]:
var example = new int[] { 3, 4, 5 };

static bool IsStrictlyIncreasing(Span<int> span)
{
    bool pass = true;
    for (
            int i = 0;
            i < span.Length - 2 + 1 && pass;
            i++
        )
        pass &= (span[i + 1] > span[i]);

    return pass;
}

IsStrictlyIncreasing(example)

True

Or format the code differently:

In [22]:
var example = new int[] { 3, 4, 5 };

static bool IsStrictlyIncreasing(Span<int> span)
{
    bool pass = true;
    for (
            int i = 0;
            i < span.Length - 2 + 1 && pass;
                pass &= (span[i + 1] > span[i]),
            i++
        );

    return pass;
}

IsStrictlyIncreasing(example)

True

I feel like every suggestion I've thrown here could make a certain type of programmer very angry, but I actually like that last one. I dislike that the iterator statement(s) in a `for` loop come before the body but are executed after. I think it leads to a lot of "off-by-one" errors for new programmers. Notably, F# doesn't have this problem because control over the iteration is either taken from you (`for...in` / `for...to`) or you are made to do it manually (`while...do`).